In [1]:
import sys

sys.path.append("/homes/gf332/compression-without-quantization/code")

import tensorflow as tf
import tensorflow_compression as tfc
import tensorflow_probability as tfp
tfd = tfp.distributions
tfk = tf.keras
tfl = tf.keras.layers

from pln import AnalysisTransform_1, SynthesisTransform_1, AnalysisTransform_2, SynthesisTransform_2, ProbabilisticLadderNetwork

tf.enable_eager_execution()

/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-pack

In [ ]:
pln = ProbabilisticLadderNetwork(first_level_filters=196,
                                 second_level_filters=128,
                                 first_level_latent_channels=128,
                                 second_level_latent_channels=24,
                                 learn_gamma=True)

test = tf.zeros((1, 256, 256, 3))

with tf.device("CPU:0"):
    pln(test)
    
    k1 = pln.first_level_kl
    
    k2 = pln.second_level_kl